In [1]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression, energy_LSTM, energy_XGBoost
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax

In [2]:
from functions import naive_ensemble

In [3]:
daxdata = get_DAX.get()

In [4]:
base = DAX_baseline.DAX_baseline(daxdata)

In [5]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,DAX,1 day,-3.321265,-0.543724,0.065019,0.710654,2.719772
1,2024-01-17,DAX,2 day,-4.087895,-0.799977,0.080265,0.976061,3.570492
2,2024-01-17,DAX,5 day,-5.259161,-0.896702,0.161798,1.224285,4.628697
3,2024-01-17,DAX,6 day,-6.282629,-1.007463,0.187498,1.435857,5.367396
4,2024-01-17,DAX,7 day,-6.735742,-1.054012,0.326190,1.557158,6.259934


In [6]:
df_dax=base

In [7]:
energydata= get_energy.get()

  0%|          | 0/265 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 265/265 [00:29<00:00,  9.08it/s]


In [8]:
df_energy= energy_XGBoost.energy_forecast(energydata,date_str='2024-01-17')

In [9]:
df_energy

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,energy,36 hour,64.057678,70.027924,70.872932,75.010704,77.218788
1,2024-01-17,energy,40 hour,61.880836,66.906807,68.240898,72.040749,73.287773
2,2024-01-17,energy,44 hour,60.779305,62.762531,64.174416,68.024254,68.785774
3,2024-01-17,energy,60 hour,57.528931,60.668816,61.805847,64.520790,64.131546
4,2024-01-17,energy,64 hour,54.591389,57.760727,58.559002,60.839371,62.414742
5,2024-01-17,energy,68 hour,53.390141,56.549107,58.772770,59.213043,59.346821


In [10]:
df = merge_submissions.merge(df_energy,df_dax)

         forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0           2024-01-17         DAX    1 day  -3.321265  -0.543724   0.065019   
1           2024-01-17         DAX    2 day  -4.087895  -0.799977   0.080265   
2           2024-01-17         DAX    5 day  -5.259161  -0.896702   0.161798   
3           2024-01-17         DAX    6 day  -6.282629  -1.007463   0.187498   
4           2024-01-17         DAX    7 day  -6.735742  -1.054012    0.32619   
0  2024-01-17 00:00:00      energy  36 hour  64.057678  70.027924  70.872932   
1  2024-01-17 00:00:00      energy  40 hour  61.880836  66.906807  68.240898   
2  2024-01-17 00:00:00      energy  44 hour  60.779305  62.762531  64.174416   
3  2024-01-17 00:00:00      energy  60 hour  57.528931  60.668816  61.805847   
4  2024-01-17 00:00:00      energy  64 hour  54.591389  57.760727  58.559002   
5  2024-01-17 00:00:00      energy  68 hour  53.390141  56.549107   58.77277   
0           2024-01-17  infections   0 w

In [12]:
import pandas as pd

In [13]:
# ensure forecast_date is in correct format
df['forecast_date'] = pd.to_datetime(df['forecast_date'], format='%Y-%m-%d')

In [14]:
df

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2024-01-17,DAX,1 day,-3.321265,-0.543724,0.065019,0.710654,2.719772
1,2024-01-17,DAX,2 day,-4.087895,-0.799977,0.080265,0.976061,3.570492
2,2024-01-17,DAX,5 day,-5.259161,-0.896702,0.161798,1.224285,4.628697
3,2024-01-17,DAX,6 day,-6.282629,-1.007463,0.187498,1.435857,5.367396
4,2024-01-17,DAX,7 day,-6.735742,-1.054012,0.32619,1.557158,6.259934
0,2024-01-17,energy,36 hour,64.057678,70.027924,70.872932,75.010704,77.218788
1,2024-01-17,energy,40 hour,61.880836,66.906807,68.240898,72.040749,73.287773
2,2024-01-17,energy,44 hour,60.779305,62.762531,64.174416,68.024254,68.785774
3,2024-01-17,energy,60 hour,57.528931,60.668816,61.805847,64.52079,64.131546
4,2024-01-17,energy,64 hour,54.591389,57.760727,58.559002,60.839371,62.414742


In [15]:
df.to_csv('../../forecasts/20240117_JonSnow.csv',index=False)

In [ ]:
df

In [ ]:
from datetime import timedelta, datetime

In [ ]:
date_str='2024-01-17'

In [ ]:
horizons = [36, 40, 44, 60, 64, 68]
base_date = datetime.strptime(date_str, '%Y-%m-%d')
#add 24 hrs to base date
base_date=base_date+timedelta(days=1)
future_timestamps = [base_date + timedelta(hours=h) for h in horizons]

In [ ]:
future_timestamps

In [ ]:
import sys
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, date

def check_df(df):
    EXPECTED_COLS = ["forecast_date", "target", "horizon", "q0.025", "q0.25",
                    "q0.5", "q0.75", "q0.975"]
    LEN_EXP_COLS = len(EXPECTED_COLS)

    # if exclude_weather == True:
    #     print("Excluding weather variables!")
    #     TARGETS = ["DAX", "energy"]
    # else:
    TARGETS = ["DAX", "energy"]

    TARGET_VALS = dict(DAX = [str(i) + " day" for i in (1,2,5,6,7)],
                    energy = [str(i) + " hour" for i in (36,40,44,60,64,68)],
                    infections = [str(i) + " week" for i in (0,1,2,3,4)])

    TARGET_LEN = dict(DAX = len(TARGET_VALS["DAX"]),
                    energy = len(TARGET_VALS["energy"]),
                    infections = len(TARGET_VALS["infections"])
                    )

    TARGET_PLAUS = dict(DAX = [-20, 20],
                        energy = [0,250],
                        infections = [0,9000])

    COLS_QUANTILES = ["q0.025", "q0.25", "q0.5", "q0.75", "q0.975"]

    print("Start checking...")
    print("---------------------------")
    col_names = df.columns


    print("Checking the Columns...")
    # Check column length
    if len(col_names) != LEN_EXP_COLS:
        print("Dataset contains ",len(col_names), "columns. Required are",LEN_EXP_COLS)
        print("Stopping early...")
        sys.exit()

    if set(col_names) != set(EXPECTED_COLS):
        print("Dataset does not contain the required columns (or more).")
        missing_cols = set(EXPECTED_COLS) - set(col_names)
        print("The missing columns are:", missing_cols)
        print("Stopping early...")
        sys.exit()

    for i,col in enumerate(EXPECTED_COLS):
        if col == col_names[i]:
            continue
        else:
            print("Columns not in correct order. Order should be:", EXPECTED_COLS)
            print("Your order is:", col_names.values)
            print("Stopping early...")
            sys.exit()

    # Date Col
    print("Checking type of columns...")
    try:
        df["forecast_date"] = pd.to_datetime(df["forecast_date"], format="%Y-%m-%d",
                                            errors="raise")
    except (pd.errors.ParserError, ValueError):
        print("Could not parse Date in format YYYY-MM-DD")
        print("Stopping early...")
        sys.exit()

    try:
        df["target"] = df["target"].astype("object", errors="raise")
    except ValueError:
        print("Cannot convert target column to String.")
        print("Stopping early...")
        sys.exit()

    try:
        df["horizon"] = df["horizon"].astype("object", errors="raise")
    except ValueError:
        print("Cannot convert horizon column to String.")
        print("Stopping early...")
        sys.exit()

    for cq in COLS_QUANTILES:
        if pd.to_numeric(df[cq], errors="coerce").isna().any():
            print("----WARNING: Some elements in",cq,"column are not numeric. This may be fine if you only submit 2 out of 3 targets.")
            print("")
            # print("Stopping early...")
            # sys.exit()

    print("Checking if the Dates make sense...")

    if len(pd.unique(df["forecast_date"])) > 1:
        print("forecast_date needs to be the same in all rows.")
        print("Stopping early...")
        sys.exit()

    if df["forecast_date"][0].date() < datetime.today().date():
        print("----WARNING: Forecast date should not be in the past.")
        print("")
        # warnings.warn("Forecast date should not be in the past.")

    if df["forecast_date"][0].weekday() != 2:
        print("----WARNING: Forecast date should be a Wednesday.")
        print("")
        # warnings.warn("Forecast date should be a Wednesday")

    print("Checking targets...")

    if not df["target"].isin(TARGETS).all():
        print(f"Target column can only contain {TARGETS}. Check spelling.")
        print("Stopping early...")
        sys.exit()

    for target in TARGETS:

        if len(df[df["target"] == target]) != TARGET_LEN[target]:
            if target == "demand":
                print("Exactly 6 rows need to have target = ", target)
            else:
                print("Exactly 5 rows need to have target =", target)
            print("Stopping early...")
            sys.exit()

        if (df[df["target"] == target]["horizon"] != TARGET_VALS[target]).any():
            print("Target", target, "horizons need to be (in this order):", TARGET_VALS[target])
            print("Stopping early...")
            sys.exit()

        if (df[df["target"] == target][COLS_QUANTILES] < TARGET_PLAUS[target][0]).any(axis=None) or \
            (df[df["target"] == target][COLS_QUANTILES] > TARGET_PLAUS[target][1]).any(axis=None):
            print("----WARNING: Implausible values for",target,"detected. You may want to re-check.")
            print("")
            # warnings.warn("Implausible values for "+str(target)+" detected. You may want to re-check them.")

    print("Checking quantiles...")

    ALL_NAN_IDX = df[df.isna().any(axis=1)].index
    NAN_TARGET_IDX_LIST = []

    if len(ALL_NAN_IDX) != 0:
        NAN_TARGET = df.iloc[ALL_NAN_IDX[0]]["target"]
        NAN_TARGET_LENS = dict(DAX = 5,
                            energy = 6,
                            infections = 5)

        NAN_TARGET_IDX_LIST = df[df["target"] == NAN_TARGET].index

        print("Assume that --",NAN_TARGET,"-- is your NaN-target. Please DOUBLECHECK if this is correct.")

        if len(ALL_NAN_IDX) > NAN_TARGET_LENS[NAN_TARGET]:
            print("Your dataframe contains more NaNs than entries for target",NAN_TARGET,".")
            print("Stopping early...")
            sys.exit()
    else:
        print("Seems like you submitted all three targets. Good job!")

    for i, row in df.iterrows():
        if i in NAN_TARGET_IDX_LIST:
            continue

        diffs = row[COLS_QUANTILES].diff()
        if diffs[1:].isna().any():
            print("Something is wrong with your quantiles.")
            print("Stopping early...")
            sys.exit()
        diffs[0] = 0
        if (diffs < 0).any():
            print("Predictive quantiles in row",i,"are not ordered correctly (need to be non-decreasing)")
            print("Stopping early...")
            sys.exit()

    print("---------------------------")
    print("Looks good!")
    
check_df(df)

In [ ]:
df